In [8]:
import depthai as dai
import cv2

# Create a pipeline
pipeline = dai.Pipeline()

# Create a color camera node and set its properties
cam_rgb = pipeline.createColorCamera()
cam_rgb.setPreviewSize(640, 480)  # Set preview size to a standard resolution
cam_rgb.setInterleaved(False)
cam_rgb.setFps(30)  # Set the camera FPS to match the display rate

# Create XLinkOut node for the camera preview
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb")
cam_rgb.preview.link(xout_rgb.input)

# Connect to the device and start the pipeline
with dai.Device(pipeline) as device:
    # Get output queue
    q_rgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

    while True:
        in_rgb = q_rgb.tryGet()  # Non-blocking call to get data from the queue
        if in_rgb is not None:
            frame = in_rgb.getCvFrame()
            cv2.imshow("RGB", frame)

            key = cv2.waitKey(1)
            if key == ord('q'):
                break
            elif key == ord('c'):  # Press 'c' to capture and save the image, press 'q' to exit.
                cv2.imwrite('captured_image.jpg', frame)
                print("Image captured and saved.")

cv2.destroyAllWindows()


Image captured and saved.
Image captured and saved.
Image captured and saved.
Image captured and saved.
Image captured and saved.
Image captured and saved.
Image captured and saved.
Image captured and saved.
Image captured and saved.


In [9]:
images = glob.glob('images/*.jpg')

for image in images:
    
    print(image)

images\img_1.jpg
images\img_10.jpg
images\img_2.jpg
images\img_3.jpg
images\img_4.jpg
images\img_5.jpg
images\img_6.jpg
images\img_7.jpg
images\img_8.jpg
images\img_9.jpg


In [10]:
import numpy as np
import cv2 as cv
import glob
import pickle



################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (9,9)
frameSize = (640,480)



# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

size_of_chessboard_squares_mm = 20
objp = objp * size_of_chessboard_squares_mm


# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


images = glob.glob('images/*.jpg')

for image in images:
    
    print(image)

    img = cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if ret == True:

        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(1000)
        
        
    print(objpoints)
    print(imgpoints)


cv.destroyAllWindows()



images\img_1.jpg
[]
[]
images\img_10.jpg
[]
[]
images\img_2.jpg
[]
[]
images\img_3.jpg
[]
[]
images\img_4.jpg
[]
[]
images\img_5.jpg
[]
[]
images\img_6.jpg
[]
[]
images\img_7.jpg
[]
[]
images\img_8.jpg
[]
[]
images\img_9.jpg
[]
[]


In [3]:

############## CALIBRATION #######################################################

ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

print("Cmera Calibrated: ", ret)
print("\nCamera Matrix:\n", cameraMatrix)
print("\nDistortion Parameters:\n", dist)
print("\n Rotation Vectors:\n", rvecs)
print("\nTranslation Vectors: \n", tvecs)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
#pickle.dump((cameraMatrix, dist), open( "calibration.pkl", "wb" ))
#pickle.dump(cameraMatrix, open( "cameraMatrix.pkl", "wb" ))
#pickle.dump(dist, open( "dist.pkl", "wb" ))



error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\calibration.cpp:3752: error: (-215:Assertion failed) nimages > 0 in function 'cv::calibrateCameraRO'


In [11]:
print("Number of object points: ", len(objpoints))
print("Number of image points: ", len(imgpoints))


Number of object points:  0
Number of image points:  0


In [5]:

############## UNDISTORTION #####################################################

img = cv.imread('cali5.png')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))



# Undistort
dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult1.png', dst)



# Undistort with Remapping
mapx, mapy = cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult2.png', dst)




# Reprojection Error
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print( "total error: {}".format(mean_error/len(objpoints)) )
print("\n\n\n")


AttributeError: 'NoneType' object has no attribute 'shape'